In [1]:
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio
import requests
import pandas as pd
import numpy as np
import os

#time
import pytz
from datetime import date, datetime, timedelta

functions compiled

In [2]:
def get_url(startTime, endTime):
    url = "https://pmmpublisher.pps.eosdis.nasa.gov/opensearch?q=precip_30mn&" + \
    "lat=30" + \
    "&" +\
    "lon=78" + \
    "&limit=&" + \
    "startTime=" + startTime + \
    "&" +\
    "endTime=" + endTime
    return url

In [3]:
def get_raster_list(url, start, end):   
    r = requests.get(url)    
    items = r.json()['items']
    turl = None
    df = pd.DataFrame(index=range(0,len(items)), columns=['date', 'url', 'inRange'])
    ind = 0
    for i in items:
        #convert date string to datetime
        date = datetime.strptime(i['properties']['date']['@value'], '%Y-%m-%dT%H:%M:%S+00:00')
        date = date.astimezone(pytz.utc) #set timezone
        df.iloc[ind, 0] = date
        
        #add info on whether or not it is in time range
        df.iloc[ind, 2] = time_in_range(start, end, date)
        
        #add urls if they are tifs
        mediaType = i['action'][1]['using'][1]['mediaType']
        if mediaType == 'image/tiff':
            df.iloc[ind, 1] = i['action'][1]['using'][1]['url']
        else:
            df.iloc[ind, 1] = i['action'][1]['using'][2]['url']
        ind += 1
    return df

In [4]:
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start < x <= end
    else:
        return start < x or x <= end

In [5]:
def get_precip_1d(df, line):
    total = 0

    for index, row in df.iterrows():
        #print(row['url'])
        src = rasterio.open(row['url'])
        #print(size(src.read(1)))
        rast_stats = zonal_stats(line, 
            src.read(1), affine=src.transform,
            stats="count min mean max median")
        mean = rast_stats[0]['mean']
        #print(rast_stats)
        total += mean
    
    return total

the road file

In [6]:
#it would be ideal to be able to feed in a road file and then it will spit out a geodataframe with the updated weather

In [7]:
os.getcwd()

'/home/jovyan/work/notebooks'

In [8]:
def add_time(file,tz):
    #assume all the closures occur or are reported at 6pm, or earlier
    dateTime = datetime.strptime(file['date'][0] + ' 17-59-59', '%Y-%m-%d %H-%M-%S')
    timezone = pytz.timezone(tz)
    
    dateTime_tz = timezone.localize(dateTime) #time in timezone
    dateTime_utc = dateTime_tz.astimezone(pytz.utc) #time in utc

    file['datetime'] = dateTime_utc
    return file, dateTime_utc

In [9]:
def add_precip_1d(file, tz):
    file, dateTime_utc = add_time(file, tz)
    
    #start and end time as strings
    endTime = dateTime_utc.strftime('%Y-%m-%d')
    startTime = (dateTime_utc - timedelta(days=1)).strftime('%Y-%m-%d')
    
    #get the urls to help get weather data !!
    url = get_url(startTime, endTime)
    df = get_raster_list(url, dateTime_utc - timedelta(days=1), dateTime_utc)
    #some ~processing~
    in_range = df.loc[df['inRange'] == True]
    try:
        file.insert(len(road.columns),'precip_1d', np.nan)
    except ValueError:
        pass
    
    #add to the list
    for index, row in road.iterrows():
        col = file.columns.get_loc("precip_1d")
        precip_1d = get_precip_1d(in_range, row['geometry'])
        file.iloc[index,col] = precip_1d
    
    print(endTime)
    
    return file

In [10]:
road = gpd.read_file('./05-methods/closure-data/2021-06-23.gpkg')
road = road.to_crs(epsg=4326)
data = add_precip_1d(road, 'Asia/Kolkata')

RasterioIOError: CURL error: error setting certificate verify locations:   CAfile: /etc/pki/tls/certs/ca-bundle.crt   CApath: none

In [ ]:
data.loc[data['precip_1d'] >0]

In [ ]:
data

In [ ]:
data.loc[data['status']==1]

In [ ]:
data.to_file('./05-methods/closure-data/2021-06-21.gpkg', driver='GPKG')

In [ ]:
os.getcwd()

In [ ]:
gpd.read_file('./05-methods/closure-data/2021-06-21.gpkg')

In [ ]:
#there's so many things, like max amount in the past day, average accumulation in the past day, idk what to do lol
#let's just stick with the average. so we're going to take the average on the road and say that's how much accumulated on that road. then add over time.

DEMS

In [ ]:

src = rasterio.open('https://pmmpublisher.pps.eosdis.nasa.gov/products/s3/r07/gpm_30mn/2021/176/gpm_30mn.20210625.082959.tif')

In [ ]:
src.crs

In [ ]:
import os
os.getcwd()

In [ ]:
src.read(1)

In [ ]:
test = zonal_stats(road.head(70)['geometry'], 
            src.read(1), affine=src.transform,
            stats="count min mean max median")

In [ ]:
#then call the necessary api for the date with the relevant inputs and stuff
url = ''

In [ ]:
#pull all the data

In [ ]:
test

In [ ]:
road